# Genius API Test

### Required libraries

In [1]:
from dotenv import load_dotenv
from lyricsgenius import Genius
import os
import pandas as pd

### Set API credentials

In [2]:
# Set up API Creds
load_dotenv()
token = os.getenv(key='CLIENT_ACCESS_TOKEN')

### Function to get lyrics by genre (tag)

In [3]:
def get_lyrics(genre, page_max, page = 1):
    
    # Loop to get lyrics by genre
    while page < page_max:
        res = genius.tag(genre, page=page)
        for hit in res['hits']:
            # Get song lyrics
            song_lyrics = genius.lyrics(song_url=hit['url'])

            # Get rid of extra info
            index = song_lyrics.find('\n')
            song_lyrics = song_lyrics[index+1:]
            lyrics.append(song_lyrics)

            # Get title
            title = hit['title']
            titles.append(title)

            # Get artists
            artist = hit['artists']
            artists.append(artist)

            # Set genre 
            genres.append(genre)

        page = res['next_page']

### Loop through genres to get data
Note we can currently pull 50 pages (page_max = 51) with 20 songs per page. We can add 50 in a loop to the 'page' variable in the 'get_lyrics' function, however ~1,000 songs a genre should suffice for now. 

In [4]:
# Set up API Access
genius = Genius(token, timeout=10, retries=3)

# Initalize lists for function
lyrics = []
titles = []
artists = []
genres = []

# Genres 
# Check out - https://genius.com/Genius-tags-music-genres-international-annotated for other tags
tags = ['country', 'rock', 'pop', 'r-b', 'rap']

# Set how many pages to pull
page_max = 3

# Loop through our genres
for genre in tags:
    print("Pulling", genre, "lyrics...")
    get_lyrics(genre=genre, page_max=page_max)

Pulling country lyrics...
Pulling rock lyrics...
Pulling pop lyrics...
Pulling r-b lyrics...


### Put everything in DF

In [25]:
df = pd.DataFrame(list(zip(artists, titles, lyrics, genres)),
                  columns=['artist', 'title', 'lyrics', 'genre'])
df.head()

,artist,title,lyrics,genre
175,[Kendrick Lamar],​m.A.A.d city,[Refrain: Kendrick Lamar & ScHoolboy Q]\nIf Pi...,rap
176,[Eminem],Killshot,"You sound like a bitch, bitch\nShut the fuck u...",rap
177,[Kanye West],Mercy,"Well, it is a weepin' and a moanin' and a gnas...",rap
178,[Drake],Hotline Bling,"You used to call me on my\nYou used to, you us...",rap
179,[XXXTENTACION],Jocelyn Flores,"I know you so well, so well\nI mean, I can do ...",rap


### Read out DF

In [31]:
df.to_csv('data/lyrics.csv.gz', compression='gzip')